# `qp` Demo

_Alex Malz & Phil Marshall_

In this notebook we use the `qp` module to approximate some simple, standard, 1-D PDFs using sets of quantiles, samples, and histograms, and assess their relative accuracy. We also show how such analyses can be extended to use "composite" PDFs made up of mixtures of standard distributions.

### Requirements

To run `qp`, you will need to first install the module. 

In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.interpolate as spi

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import qp

## The `qp.PDF` Class

This is the basic element of `qp` - an object representing a probability density function. This class is stored in the module `pdf.py`.  The PDF must be initialized with some representation of the distribution.

In [ ]:
# ! cat qp/pdf.py
#P = qp.PDF(vb=True)

## Approximating a Gaussian

Let's summon a PDF object, and initialize it with a standard function - a Gaussian.

In [ ]:
limits = (-5., 5.)
dist = sps.norm(loc=0, scale=1)
print(type(dist))
qp.qp_plot_native(None, dist, label="pdf", limits=limits)

### Samples

Let's sample the PDF to see how it looks.  When we plot the `PDF` object, both the true and sampled distributions are displayed.

In [ ]:
np.random.seed(42)

kde_dist = qp.qp_convert(dist, qp.kde_dist)
qp.qp_plot_native(None, kde_dist, limits=limits)

### Quantile Parametrization

Now, let's compute a set of evenly spaced quantiles. These will be carried by the `PDF` object as `p.quantiles`.  We also demonstrate the initialization of a `PDF` object with quantiles and no truth function.

In [ ]:
quantiles = np.linspace(0.01, 0.99, 11)

quant_dist = qp.qp_convert(dist, qp.quantile_dist, quantiles=quantiles)
qp.qp_plot_native(None, quant_dist, limits=limits)

### Histogram Parametrization

Let's also compute a histogram representation, that will be carried by the `PDF` object as `p.histogram`. The values in each bin are the integrals of the PDF over the range defined by bin ends. We can also initialize a `PDF` object with a histogram and no truth function.

In [ ]:
bins = np.linspace(-5, 5, 11)
hist_dist = qp.qp_convert(dist, sps.rv_histogram, bins=bins)
qp.qp_plot_native(None, hist_dist, limits=limits)

### Evaluating the Approximate PDF by Interpolation

Once we have chosen a parametrization to approximate the PDF with, we can evaluate the approximate PDF at any point by interpolation (or extrapolation). `qp` uses [`scipy.intepolate.interp1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) to do this, with `linear` as the default interpolation scheme. (Most other options do not enable extrapolation, `nearest` being the exception.)

Let's test this interpolation by evaluating an approximation at a single point using the quantile parametrization.

In [ ]:
print (qp.approximate_dist(quant_dist, np.array([0.314])))

In [ ]:
dist.pdf(np.array([0.314]))

(We can also integrate any approximation.)

In [ ]:
print (qp.integrate_dist(quant_dist, limits=(0,1)))

We can also interpolate the function onto an evenly spaced grid with points within and out of the quantile range, as follows:  

In [ ]:
grid = np.linspace(-3., 3., 100)
gridded = quant_dist.pdf(grid)
print(gridded)

We can also change the interpolation scheme:

### Mixture Model Fit

We can fit a parametric mixture model to samples from any parametrization.  Currently, only a Gaussian mixture model is supported.

In [ ]:
mixmod_dist = qp.qp_convert(dist, qp.sum_dist, ncomps=3, nsamples=1000)
qp.qp_plot_native(None, mixmod_dist, limits=limits, label="mixmod")

## Comparing Parametrizations

`qp` supports both qualitative and quantitative comparisons between different distributions, across parametrizations.

### Qualitative Comparisons: Plotting

Let's visualize the PDF object in order to compare the truth and the approximations.  The solid, black line shows the true PDF evaluated between the bounds.  The green rugplot shows the locations of the 1000 samples we took.  The vertical, dotted, blue lines show the percentiles we asked for, and the hotizontal, dotted, red lines show the 10 equally spaced bins we asked for.  Note that the quantiles refer to the probability distribution *between the bounds*, because we are not able to integrate numerically over an infinite range. Interpolations of each parametrization are given as dashed lines in their corresponding colors.  Note that the interpolations of the quantile and histogram parametrizations are so close to each other that the difference is almost imperceptible!

In [ ]:
fig, axes = qp.make_figure_axes(limits=limits)
for dist, label in zip([dist, hist_dist, quant_dist, kde_dist, mixmod_dist], ["Truth", "Hist", "Quantiles", "KDE", "Mixture"]):
    qp.qp_plot_native(axes, dist)    

### Quantitative Comparisons

In [ ]:
symm_lims = np.array([-1., 1.])
all_lims = [symm_lims, 2.*symm_lims, 3.*symm_lims]

Next, let's compare the different parametrizations to the truth using the Kullback-Leibler Divergence (KLD).  The KLD is a measure of how close two probability distributions are to one another -- a smaller value indicates closer agreement.  It is measured in units of bits of information, the information lost in going from the second distribution to the first distribution.  The KLD calculator here takes in a shared grid upon which to evaluate the true distribution and the interpolated approximation of that distribution and returns the KLD of the approximation relative to the truth, which is not in general the same as the KLD of the truth relative to the approximation.  Below, we'll calculate the KLD of the approximation relative to the truth over different ranges, showing that it increases as it includes areas where the true distribution and interpolated distributions diverge.

In [ ]:
for PDF in [hist_dist, quant_dist, kde_dist, mixmod_dist]:
    D = []
    for lims in all_lims:
        D.append(qp.metrics.calculate_kld(dist, PDF, limits=lims, vb=False))
    print(str(type(PDF))+' approximation: KLD over 1, 2, 3, sigma ranges = '+str(D))

Holy smokes, does the quantile approximation blow everything else out of the water, thanks to using spline interpolation.

The progression of KLD values should follow that of the root mean square error (RMSE), another measure of how close two functions are to one another.  The RMSE also increases as it includes areas where the true distribution and interpolated distribution diverge.  Unlike the KLD, the RMSE is symmetric, meaning the distance measured is not that of one distribution from the other but of the symmetric distance between them.

In [ ]:
for PDF in [hist_dist, quant_dist, kde_dist, mixmod_dist]:
    D = []
    for lims in all_lims:
        D.append(qp.metrics.calculate_rmse(dist, PDF, limits=lims, vb=False))
    print(str(type(PDF))+' approximation: RMSE over 1, 2, 3, sigma ranges = '+str(D))

Both the KLD and RMSE metrics suggest that the quantile approximation is better in the high density region, but samples work better when the tails are included. We might expect the answer to the question of which approximation to use to depend on the application, and whether the tails need to be captured or not.

Finally, we can compare the meoments of each approximation and compare those to the moments ofthe true distribution.

In [ ]:
pdfs = [dist, hist_dist, quant_dist, kde_dist, mixmod_dist]
which_moments = range(3)
all_moments = []
for pdf in pdfs:
    moments = []
    for n in which_moments:
        moments.append(qp.metrics.calculate_moment(pdf, n, limits=limits))
    all_moments.append(moments)
    
print('moments: '+str(which_moments))
for pdf, moments in zip(pdfs, all_moments):
    print(str(type(pdf))+': '+str(moments))

The first three moments have an interesting interpretation.  The zeroth moment should always be 1 when calculated over the entire range of redshifts, but the quantile approximation is off by about $7\%$.  We know the first moment in this case is 0, and indeed the evaluation of the first moment for the true distribution deviates from 0 by less than Python's floating point precision.  The samples parametrization has a biased estimate for the first moment to the tune of $2\%$.  The second moment for the true distribution is 1, and the quantile parametrization (and, to a lesser extent, the histogram parametrization) fails to provide a good estimate of it.

## Advanced Usage

### Composite PDFs

In addition to individual `scipy.stats.rv_continuous` objects, `qp` can be initialized with true distributions that are linear combinations of `scipy.stats.rv_continuous` objects.  To do this, one must create the component distributions and specify their relative weights.  This can be done by running `qp.PDF.mix_mod_fit()` on an existing `qp.PDF` object once samples have been calculated, or it can be done by hand.

In [ ]:
dists = [sps.norm(loc=-2., scale=1.), sps.norm(loc=2., scale=1.)]
coefs = [0.8, 0.2]


comp_dist = qp.sum_dist(coefs=coefs, dists=dists)
qp.qp_plot_native(None, comp_dist, label="mixmod", limits=limits)

We can calculate the quantiles for such a distribution.

In [ ]:
Cq = qp.qp_convert(comp_dist, qp.quantile_dist, quantiles=quantiles)
qp.qp_plot_native(None, Cq, limits=limits)

Similarly, the histogram parametrization is also supported for composite PDFs.

In [ ]:
hist = qp.utils.histogramize_dist(comp_dist, bins=bins)
print(hist)
Ch = sps.rv_histogram((hist[1],hist[0]))

In [ ]:
Ch = qp.qp_convert(comp_dist, sps.rv_histogram, bins=bins)
qp.qp_plot_native(None, Ch, limits=limits)

Finally, samples from this distribution may also be taken, and a PDF may be reconstructed from them.  Note: this uses [`scipy.stats.gaussian_kde`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html), which determines its bandwidth/kernel size using Scott's Rule, Silverman's Rule, a fixed bandwidth, or a callable function that returns a bandwidth.

In [ ]:
samples = comp_dist.rvs(size=1000)
Cs = qp.kde_dist(samples)

In [ ]:
qD = qp.metrics.calculate_kld(comp_dist, Cq, limits=limits, dx=0.001, vb=True)
hD = qp.metrics.calculate_kld(comp_dist, Ch, limits=limits, dx=0.001, vb=True)
sD = qp.metrics.calculate_kld(comp_dist, Cs, limits=limits, dx=0.001, vb=True)
print(qD, hD, sD)

### PDF Ensembles

`qp` also includes infrastructure for handling ensembles of `PDF` objects with shared metaparameters, such as histogram bin ends, but unique per-object parameters, such as histogram bin heights.  A `qp.Ensemble` object takes as input the number of items in the ensemble and, optionally, a list, with contents corresponding to one of the built-in formats.

Let's demonstrate on PDFs with a functional form, which means the list of information for each member of the ensemble is `scipy.stats.rv_continuous` or `qp.composite` objects.

In [ ]:
N = 25
means = sps.uniform.rvs(size=N)
stdevs = 1.0 + 0.5*sps.uniform.rvs(size=N)
dists = [sps.norm(loc=mean, scale=stdev) for mean, stdev in zip(means, stdevs)]
E = qp.Ensemble(dists)

As with individual `qp.PDF` objects, we can evaluate the PDFs at given points, convert to other formats, and integrate.

In [ ]:
eval_range = np.linspace(-5., 5., 100)
E.evaluate(eval_range)

In [ ]:
E.quantize(quantiles)

In [ ]:
E.integrate(limits=(0., 1.))

Previous versions of `qp` included a built-in function for "stacking" the member PDFs of a `qp.Ensemble` object.  This functionality has been removed to discourage use of this procedure in science applications.  However, we provide a simple function one may use should this functionality be desired.

In [ ]:
E.dist_class

In [ ]:
E_hist = qp.qp_convert(E, sps.rv_histogram, bins=bins)

In [ ]:
def stack(ensemble, loc):
    """
    Produces an average of the PDFs in the ensemble

    Parameters
    ----------
    ensemble: qp.Ensemble
        the ensemble of PDFs to stack
    loc: ndarray, float or float
        location(s) at which to evaluate the PDFs
    using: string
        which parametrization to use for the approximation
    vb: boolean
        report on progress

    Returns
    -------
    stacked: tuple, ndarray, float
        pair of arrays for locations where approximations were evaluated
        and the values of the stacked PDFs at those points
    """
    evaluated = ensemble.evaluate(loc)
    stack = np.mean(evaluated[-1], axis=0)
    stacked = (loc, stack)
    return stacked

In [ ]:
stacked = stack(E, eval_range)
plt.plot(stacked[0], stacked[1])